# Summarization of financial data using Hugging Face LLM models
This notebook aims to provide an introduction to documenting an LLM model using the ValidMind Developer Framework. The use case presented is a summarization of financial data (https://huggingface.co/datasets/financial_phrasebank).

- Initializing the ValidMind Developer Framework
- Running a test various tests to quickly generate document about the data and model

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
# %pip install --upgrade validmind

## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace the code below with the code snippet from your project ## 

# import validmind as vm

# vm.init(
#  api_key = "2494c3838f48efe590d531bfe225d90b",
#  api_host = "http://localhost:3000/api/v1/tracking",
#  api_secret = "4f692f8161f128414fef542cab2a4e74834c75d01b3a8e088a1834f2afcfe838",
#  project = "cllnq0ckr000273y6ev40pmb5"
#)
  

## Use case

In the realm of financial news, accurate interpretation of information is key. Given the vast amounts of data, we use Hugging Face's LLMs for summarization of financial articles. This notebook focuses on evaluating these text summarization models, especially using validation metrics. These metrics don't just measure the data and model's performance and accuracy; they also check for any bias or toxicity in the summaries. This helps ensures developers and users that the summarized content is both trustworthy and compliant with AI principles.

## 1. Setup

### Import Libraries

In [3]:
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import textwrap
from tabulate import tabulate
from IPython.display import display, HTML
from rouge import Rouge
import plotly.graph_objects as go
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from pprint import pprint
import torch
import string
import plotly.express as px
import plotly.subplots as sp
from collections import Counter
from itertools import combinations
from dataclasses import dataclass

### Preprocessing functions

In [4]:
def add_id_column(df):
    """
    Adds an 'ID' column to the dataframe.
    
    Args:
    - df (pd.DataFrame): The input dataframe.
    
    Returns:
    - pd.DataFrame: The dataframe with an added 'ID' column.
    """
    df.insert(0, 'ID', range(1, 1 + len(df)))
    return df

def load_text_data(filepath, num_records=5):
    """
    Load a CSV file, limit the number of records, and add an 'ID' column.
    
    Args:
    - filepath (str): Path to the CSV file.
    - num_records (int): Number of records to load.
    
    Returns:
    - pd.DataFrame: The dataframe with the specified number of records and an added 'ID' column.
    """
    df = pd.read_csv(filepath)
    df = df.head(num_records)
    return add_id_column(df)

In [5]:
import json
import pandas as pd

def load_toxic_prompt_data(file_path, threshold=None):
    """
    Reads a .jsonl file and returns a dataframe with all records.

    :param file_path: Path to the .jsonl file.
    :param threshold: Optional toxicity threshold. If provided, only records with a toxicity
                      score exceeding this threshold will be returned.
    :return: DataFrame with all records.
    """
    # Load records
    records = []
    with open(file_path, "r") as file:
        for line in file:
            data = json.loads(line)
            records.append(data)
            
    # Prepare data for dataframe
    data_list = []
    for record in records:
        prompt_toxicity = record["prompt"].get("toxicity", None)
        cont_toxicity = record["continuation"].get("toxicity", None)
        
        if threshold:
            if not prompt_toxicity or not cont_toxicity:
                continue  # If we don't have a toxicity value, skip the record
            elif prompt_toxicity < threshold and cont_toxicity < threshold:
                continue  # If neither toxicity value exceeds the threshold, skip the record

        row = {
            "Filename": record["filename"],
            "Begin": record["begin"],
            "End": record["end"],
            "Challenging": record["challenging"],
            "Prompt Text": record["prompt"]["text"],
            "Prompt Profanity": record["prompt"].get("profanity", "N/A"),
            "Prompt Sexually Explicit": record["prompt"].get("sexually_explicit", "N/A"),
            "Prompt Identity Attack": record["prompt"].get("identity_attack", "N/A"),
            "Prompt Flirtation": record["prompt"].get("flirtation", "N/A"),
            "Prompt Threat": record["prompt"].get("threat", "N/A"),
            "Prompt Insult": record["prompt"].get("insult", "N/A"),
            "Prompt Severe Toxicity": record["prompt"].get("severe_toxicity", "N/A"),
            "Prompt Toxicity": record["prompt"].get("toxicity", "N/A"),
            "Continuation Text": record["continuation"]["text"],
            "Cont. Severe Toxicity": record["continuation"].get("severe_toxicity", "N/A"),
            "Cont. Toxicity": record["continuation"].get("toxicity", "N/A"),
            "Cont. Profanity": record["continuation"].get("profanity", "N/A"),
            "Cont. Sexually Explicit": record["continuation"].get("sexually_explicit", "N/A"),
            "Cont. Identity Attack": record["continuation"].get("identity_attack", "N/A"),
            "Cont. Flirtation": record["continuation"].get("flirtation", "N/A"),
            "Cont. Threat": record["continuation"].get("threat", "N/A"),
            "Cont. Insult": record["continuation"].get("insult", "N/A")
        }
        data_list.append(row)

    # Convert list of dicts to dataframe
    df = pd.DataFrame(data_list)
    
    return df


In [6]:
def _format_cell_text(text, width=50):  
    """Private function to format a cell's text."""
    return '\n'.join([textwrap.fill(line, width=width) for line in text.split('\n')])

def _format_dataframe_for_tabulate(df):
    """Private function to format the entire DataFrame for tabulation."""
    df_out = df.copy()
    
    # Format all string columns
    for column in df_out.columns:
        if df_out[column].dtype == object:  # Check if column is of type object (likely strings)
            df_out[column] = df_out[column].apply(_format_cell_text)
    return df_out

def _dataframe_to_html_table(df):
    """Private function to convert a DataFrame to an HTML table."""
    headers = df.columns.tolist()
    table_data = df.values.tolist()
    return tabulate(table_data, headers=headers, tablefmt="html")

def display_nice(df, num_rows=None):
    """Primary function to format and display a DataFrame."""
    if num_rows is not None:
        df = df.head(num_rows)
    formatted_df = _format_dataframe_for_tabulate(df)
    html_table = _dataframe_to_html_table(formatted_df)
    display(HTML(html_table))


In [7]:
def add_list_to_df(df, column_data, column_name):
    """
    Adds a new column to the dataframe df that contains the given data.

    Parameters:
    - df (pd.DataFrame): The original pandas DataFrame.
    - column_data (list/array): List/array of data to be added as a new column.
    - column_name (str): Name of the new column.

    Returns:
    - pd.DataFrame: DataFrame with an additional column.
    """
    
    df = df.copy()  # Make an explicit copy of the DataFrame

    # Check if the length of column_data matches the number of rows in the DataFrame
    if len(column_data) != len(df):
        raise ValueError(f"The length of column_data ({len(column_data)}) does not match the number of rows in the DataFrame ({len(df)}).")

    # Add the column_data to the DataFrame
    df[column_name] = column_data

    return df


In [8]:
def add_summaries_to_df(df, summaries):
    """
    Adds a new column 'summary_X' to the dataframe df that contains the given summaries, where X is an incremental number.

    Parameters:
    - df: The original pandas DataFrame.
    - summaries: List/array of summarized texts.

    Returns:
    - A new DataFrame with an additional summary column, with 'labels' being the first column followed by the original 'text'.
    """

    df = df.copy()  # Make an explicit copy of the DataFrame

    # Check if the length of summaries matches the number of rows in the DataFrame
    if len(summaries) != len(df):
        raise ValueError(f"The number of summaries ({len(summaries)}) does not match the number of rows in the DataFrame ({len(df)}).")

    # Determine the name for the new summary column
    col_index = 1
    col_name = 'summary_1'
    while col_name in df.columns:
        col_index += 1
        col_name = f'summary_{col_index}'

    # Add the summaries to the DataFrame
    df[col_name] = summaries

    # Rearrange the DataFrame columns to have 'topic' first, then the original 'input', followed by summary columns
    summary_columns = [col for col in df.columns if col.startswith('summary')]
    other_columns = [col for col in df.columns if col not in summary_columns + ['topic', 'input', 'reference_summary']]
    
    columns_order = ['topic', 'input', 'reference_summary'] + sorted(summary_columns) + other_columns
    df = df[columns_order]

    return df


### POC Validation Metrics

In [9]:
import evaluate 
toxicity = evaluate.load("toxicity")

Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint


In [10]:
import plotly.graph_objects as go

def hf_toxicity_plot(df, params):
    """
    Compute toxicity scores for texts and then plot line plots for input and generated texts 
    where the generated text score surpasses the given threshold.

    Parameters:
    - df (pd.DataFrame): The dataframe containing texts.
    - params (dict): Parameters containing toxicity evaluation object, column names, and generated score threshold.
    """
    
    # Extract necessary parameters
    toxicity = params["hf_toxicity_obj"]
    input_text_col = params["input_text_col"]
    generated_text_col = params["generated_text_col"]
    generated_score_threshold = params.get("generated_score_threshold", 0)  # default to 0 if not provided

    # Get list of texts from dataframe
    input_texts = df[input_text_col].tolist()
    generated_texts = df[generated_text_col].tolist()

    # Compute toxicity for input texts and generated texts
    input_toxicity_scores = toxicity.compute(predictions=input_texts)['toxicity']
    generated_toxicity_scores = toxicity.compute(predictions=generated_texts)['toxicity']

    # Filter records where the generated score is above the threshold
    indices = [i for i, gen_score in enumerate(generated_toxicity_scores) if gen_score > generated_score_threshold]

    input_toxicity_scores = [score for i, score in enumerate(input_toxicity_scores) if i in indices]
    generated_toxicity_scores = [score for i, score in enumerate(generated_toxicity_scores) if i in indices]

    # Create figure
    fig = go.Figure()

    # Add traces for the scores with modified line width
    fig.add_trace(go.Scatter(x=indices, y=input_toxicity_scores, mode='lines+markers', name='Input Text',
                             line=dict(width=1)))  # Set width to 1 for a thinner line
    fig.add_trace(go.Scatter(x=indices, y=generated_toxicity_scores, mode='lines+markers', name='Generated Text',
                             line=dict(width=1)))  # Set width to 1 for a thinner line

    # Add a trace for the threshold to appear in the legend (without actual data)
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'Threshold ({generated_score_threshold})',
                             line=dict(color="grey", width=0.5, dash="dash")))

    # Add a horizontal line for the threshold
    fig.add_shape(
        go.layout.Shape(
            type="line",
            x0=min(indices) if indices else 0,
            x1=max(indices) if indices else 1,
            y0=generated_score_threshold,
            y1=generated_score_threshold,
            line=dict(color="grey", width=0.8, dash="dash")
        )
    )

    # Update layout
    fig.update_layout(title="Toxicity Scores for Input and Generated Texts with Generated Score above threshold",
                      xaxis_title="Index",
                      yaxis_title="Toxicity Score",
                      legend_title="Text Type")

    # Show figure
    fig.show()

In [11]:
def hf_toxicity_table(df, params):
    """
    Update and return dataframe with toxicity scores for prompt and continuation texts.

    Parameters:
    - df (pd.DataFrame): The dataframe containing texts.
    - params (dict): Parameters containing toxicity evaluation object, column names, and the max and min generated toxicity thresholds.

    Returns:
    - pd.DataFrame: Updated dataframe with toxicity scores.
    """
    
    df = df.copy()  # Create a deep copy of the DataFrame
    
    # Extract necessary parameters
    toxicity = params["hf_toxicity_obj"]
    input_text_col = params["input_text_col"]
    generated_text_col = params["generated_text_col"]
    max_generated_toxicity_threshold = params.get("max_generated_toxicity_threshold", 1)  # default to 1 if not provided
    min_generated_toxicity_threshold = params.get("min_generated_toxicity_threshold", 0)  # default to 0 if not provided

    # Get list of texts from dataframe
    input_texts = df[input_text_col].tolist()
    generated_texts = df[generated_text_col].tolist()

    # Compute toxicity for input texts and generated texts
    input_toxicity_scores = toxicity.compute(predictions=input_texts)['toxicity']
    generated_toxicity_scores = toxicity.compute(predictions=generated_texts)['toxicity']

    # Assign the new toxicity scores to the dataframe using .loc to avoid the warning
    df.loc[:, "Input Text Toxicity"] = input_toxicity_scores
    df.loc[:, "Generated Text Toxicity"] = generated_toxicity_scores

    # Filter the dataframe to return only rows where the generated text toxicity score is between the thresholds
    df = df[(df["Generated Text Toxicity"] >= min_generated_toxicity_threshold) &
            (df["Generated Text Toxicity"] <= max_generated_toxicity_threshold)]

    # Order the results by "Generated Text Toxicity" in descending order
    df = df.sort_values(by="Generated Text Toxicity", ascending=False)

    return df


In [12]:
import plotly.graph_objects as go
import plotly.subplots as sp

def hf_toxicity_histograms(df, params):
    """
    Compute toxicity scores for texts and then plot histograms for input and generated texts.

    Parameters:
    - df (pd.DataFrame): The dataframe containing texts.
    - params (dict): Parameters containing toxicity evaluation object and column names.
    """
    
    # Extract necessary parameters
    toxicity = params["hf_toxicity_obj"]
    input_text_col = params["input_text_col"]
    generated_text_col = params["generated_text_col"]

    # Get list of texts from dataframe
    input_texts = df[input_text_col].tolist()
    generated_texts = df[generated_text_col].tolist()

    # Compute toxicity for input texts and generated texts
    input_toxicity_scores = toxicity.compute(predictions=input_texts)['toxicity']
    generated_toxicity_scores = toxicity.compute(predictions=generated_texts)['toxicity']

    # Create a subplot layout
    fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Input Text Toxicity", "Generated Text Toxicity"))

    # Add traces
    fig.add_trace(go.Histogram(x=input_toxicity_scores, name="Input Text"), row=1, col=1)
    fig.add_trace(go.Histogram(x=generated_toxicity_scores, name="Generated Text"), row=1, col=2)

    # Update layout
    fig.update_layout(title_text="Histograms of Toxicity Scores")
    fig.update_xaxes(title_text="Toxicity Score", row=1, col=1)
    fig.update_xaxes(title_text="Toxicity Score", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=1, col=1)
    fig.update_yaxes(title_text="Frequency", row=1, col=2)

    # Show figure
    fig.show()

In [13]:
# Secondary functions
def general_text_metrics(df, text_column):
    nltk.download('punkt', quiet=True)
    
    results = []

    for text in df[text_column]:
        sentences = nltk.sent_tokenize(text)
        words = nltk.word_tokenize(text)
        paragraphs = text.split("\n\n")

        total_words = len(words)
        total_sentences = len(sentences)
        avg_sentence_length = round(sum(len(sentence.split()) for sentence in sentences) / total_sentences if total_sentences else 0, 1)
        total_paragraphs = len(paragraphs)

        results.append([total_words, total_sentences, avg_sentence_length, total_paragraphs])

    return pd.DataFrame(results, columns=["Total Words", "Total Sentences", "Avg Sentence Length", "Total Paragraphs"])

def vocabulary_structure_metrics(df, text_column, unwanted_tokens, lang):
    stop_words = set(word.lower() for word in stopwords.words(lang))
    unwanted_tokens = set(token.lower() for token in unwanted_tokens)

    results = []

    for text in df[text_column]:
        words = nltk.word_tokenize(text)

        filtered_words = [word for word in words if word.lower() not in stop_words and word.lower() not in unwanted_tokens and word not in string.punctuation]

        total_unique_words = len(set(filtered_words))
        total_punctuations = sum(1 for word in words if word in string.punctuation)
        lexical_diversity = round(total_unique_words / len(filtered_words) if filtered_words else 0, 1)

        results.append([total_unique_words, total_punctuations, lexical_diversity])

    return pd.DataFrame(results, columns=["Total Unique Words", "Total Punctuations", "Lexical Diversity"])

# Primary function
def text_description_table(df, params):
    text_column = params["text_column"]
    unwanted_tokens = params["unwanted_tokens"]
    lang = params["lang"]
    
    gen_metrics_df = general_text_metrics(df, text_column)
    vocab_metrics_df = vocabulary_structure_metrics(df, text_column, unwanted_tokens, lang)
    
    combined_df = pd.concat([gen_metrics_df, vocab_metrics_df], axis=1)
    
    return combined_df


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import nltk
import pandas as pd
from nltk.corpus import stopwords
import string

# Ensuring NLTK resources are downloaded
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

def text_description_histograms(df, params):
    """
    This function takes a dataframe and plots histograms for the specified metrics.

    Parameters:
    - df (DataFrame): DataFrame containing the text data.
    - params (dict): Dictionary containing parameters like "text_column", "unwanted_tokens", and "lang".
    
    Returns:
    - Plotly figure containing the histograms.
    """
    
    text_column = params["text_column"]
    
    # Combined function to get all metrics
    gen_metrics_df = general_text_metrics(df, text_column)
    vocab_metrics_df = vocabulary_structure_metrics(df, text_column, params["unwanted_tokens"], params["lang"])
    combined_df = pd.concat([gen_metrics_df, vocab_metrics_df], axis=1)
    
    # Determine number of rows based on number of metrics
    num_rows = (combined_df.shape[1] + 1) // 2  # Ceiling division to ensure every metric gets a subplot
    
    # Create subplots
    fig = make_subplots(rows=num_rows, cols=2, subplot_titles=combined_df.columns)

    # For each metric, plot a histogram
    for index, column in enumerate(combined_df.columns):
        row, col = divmod(index, 2)
        fig.add_trace(
            go.Histogram(x=combined_df[column], name=column),
            row=row+1,
            col=col+1
        )

    # Update layout for better appearance and adjust the height
    subplot_height = 400  # Define the height of each individual subplot
    total_height = num_rows * subplot_height
    fig.update_layout(title_text="Distribution of Text Metrics", bargap=0.2, bargroupgap=0.1, height=total_height)
    
    return fig.show()

In [14]:
def text_structure_histograms(df, params):
    
    text_column = params["text_column"]
    num_docs_to_plot = params["num_docs_to_plot"]
    
    
    # Ensure the nltk punkt tokenizer is downloaded
    nltk.download('punkt', quiet=True)
    
    # Decide on the number of documents to plot
    if not num_docs_to_plot or num_docs_to_plot > len(df):
        num_docs_to_plot = len(df)

    # Colors for each subplot
    colors = ['blue', 'green', 'red', 'purple']

    # Axis titles for clarity
    x_titles = [
        "Word Frequencies",
        "Sentence Position in Document",
        "Sentence Lengths (Words)",
        "Word Lengths (Characters)"
    ]
    y_titles = [
        "Number of Words",
        "Sentence Length (Words)",
        "Number of Sentences",
        "Number of Words"
    ]

    # Iterate over each document in the DataFrame up to the user-specified limit
    for index, (idx, row) in enumerate(df.head(num_docs_to_plot).iterrows()):
        # Create subplots with a 2x2 grid for each metric
        fig = sp.make_subplots(
            rows=2, cols=2, 
            subplot_titles=[
                "Word Frequencies", 
                "Sentence Positions",
                "Sentence Lengths", 
                "Word Lengths"
            ]
        )
        
        # Tokenize document into sentences and words
        sentences = nltk.sent_tokenize(row[text_column])
        words = nltk.word_tokenize(row[text_column])
        
        # Metrics computation
        word_freq = Counter(words)
        freq_counts = Counter(word_freq.values())
        word_frequencies = list(freq_counts.keys())
        word_frequency_counts = list(freq_counts.values())
        
        sentence_positions = list(range(1, len(sentences) + 1))
        sentence_lengths = [len(sentence.split()) for sentence in sentences]
        word_lengths = [len(word) for word in words]
        
        # Adding data to subplots
        fig.add_trace(go.Bar(x=word_frequencies, y=word_frequency_counts, marker_color=colors[0], showlegend=False), row=1, col=1)
        fig.add_trace(go.Bar(x=sentence_positions, y=sentence_lengths, marker_color=colors[1], showlegend=False), row=1, col=2)
        fig.add_trace(go.Histogram(x=sentence_lengths, nbinsx=50, opacity=0.75, marker_color=colors[2], showlegend=False), row=2, col=1)
        fig.add_trace(go.Histogram(x=word_lengths, nbinsx=50, opacity=0.75, marker_color=colors[3], showlegend=False), row=2, col=2)

        # Update x and y axis titles
        for i, (x_title, y_title) in enumerate(zip(x_titles, y_titles)):
            fig['layout'][f'xaxis{i+1}'].update(title=x_title, titlefont=dict(size=10))
            fig['layout'][f'yaxis{i+1}'].update(title=y_title, titlefont=dict(size=10))

        # Update layout
        fig.update_layout(
            title=f"Text Description for Document {index+1}",
            barmode='overlay',
            height=800
        )
        
        fig.show()

In [34]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming the previously defined functions and parameters are present

def text_description_scatter(df, params):
    """
    This function takes a dataframe and plots scatter plots for the specified combinations of metrics.

    Parameters:
    - df (DataFrame): DataFrame containing the text data.
    - params (dict): Dictionary containing parameters like "combinations_to_plot", "text_column", "unwanted_tokens", and "lang".
    
    Returns:
    - Plotly figure containing the scatter plots.
    """
    
    text_column = params["text_column"]
    
    # Combined function to get all metrics
    gen_metrics_df = general_text_metrics(df, text_column)
    vocab_metrics_df = vocabulary_structure_metrics(df, text_column, params["unwanted_tokens"], params["lang"])
    combined_df = pd.concat([gen_metrics_df, vocab_metrics_df], axis=1)
    
    combinations_to_plot = params["combinations_to_plot"]
    num_combinations = len(combinations_to_plot)
    
    # Determine number of rows based on number of combinations
    num_rows = (num_combinations + 1) // 2  # Ceiling division
    
    # Create subplots
    fig = make_subplots(rows=num_rows, cols=2, subplot_titles=[f"{x} vs {y}" for x, y in combinations_to_plot])
    
    # For each combination, plot a scatter plot
    for index, (x_column, y_column) in enumerate(combinations_to_plot):
        row, col = divmod(index, 2)
        fig.add_trace(
            go.Scatter(x=combined_df[x_column], y=combined_df[y_column], mode='markers', showlegend=False),
            row=row+1,
            col=col+1
        )
        # Update axis titles
        fig.update_xaxes(title_text=x_column, row=row+1, col=col+1)
        fig.update_yaxes(title_text=y_column, row=row+1, col=col+1)

    # Update layout for better appearance
    subplot_height = 400  # Define the height of each individual subplot
    total_height = num_rows * subplot_height
    fig.update_layout(title_text="Scatter Plots of Text Metrics Combinations", height=total_height)
    
    return fig.show()

# You can call the function with:
# text_description_scatter(df, params)


In [16]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from transformers import BertTokenizer

def token_disparity_histograms(df, params):
    """
    Visualize the token counts distribution of two given columns using histograms.
    
    :param df: DataFrame containing the text columns.
    :param params: Dictionary with the keys ["reference_column", "generated_column"].
    """
    
    reference_column = params["reference_column"]
    generated_column = params["generated_column"]
    
    # Initialize the tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Tokenize the columns and get the number of tokens
    df['tokens_1'] = df[reference_column].apply(lambda x: len(tokenizer.tokenize(x)))
    df['tokens_2'] = df[generated_column].apply(lambda x: len(tokenizer.tokenize(x)))
    
    # Create subplots: 1 row, 2 columns
    fig = make_subplots(rows=1, cols=2, subplot_titles=(f'Tokens in {reference_column}', f'Tokens in {generated_column}'))

    # Add histograms
    fig.add_trace(go.Histogram(x=df['tokens_1'], 
                               marker_color='blue',
                               name=f'Tokens in {reference_column}'),
                  row=1, col=1)

    fig.add_trace(go.Histogram(x=df['tokens_2'], 
                               marker_color='red',
                               name=f'Tokens in {generated_column}'),
                  row=1, col=2)

    # Update layout
    fig.update_layout(title_text='Token Distributions',
                      bargap=0.1)

    fig.update_yaxes(title_text='Number of Documents')
    fig.update_xaxes(title_text='Number of Tokens', row=1, col=1)
    fig.update_xaxes(title_text='Number of Tokens', row=1, col=2)

    fig.show()



In [17]:
from rouge import Rouge
import pandas as pd
import plotly.graph_objects as go

def rouge_scores_plot(df, params):
    """
    Compute ROUGE scores for each row in the DataFrame and visualize them.
    
    :param df: DataFrame containing the summaries.
    :param params: Dictionary with the keys ["metric", "ref_column", "gen_column"].
    """
    
    # Extract parameters
    metric = params.get("metric", "rouge-2")
    ref_column = params["ref_column"]
    gen_column = params["gen_column"]
    
    if metric not in ["rouge-1", "rouge-2", "rouge-l", "rouge-s"]:
        raise ValueError("Invalid metric. Choose from 'rouge-1', 'rouge-2', 'rouge-l', 'rouge-s'.")
    
    rouge = Rouge(metrics=[metric])
    score_list = []
    
    for _, row in df.iterrows():
        scores = rouge.get_scores(row[gen_column], row[ref_column], avg=True)[metric]
        score_list.append(scores)
    
    df_scores = pd.DataFrame(score_list)
    
    # Visualization part
    fig = go.Figure()

    # Adding the line plots
    fig.add_trace(go.Scatter(x=df_scores.index, y=df_scores['p'], mode='lines+markers', name='Precision'))
    fig.add_trace(go.Scatter(x=df_scores.index, y=df_scores['r'], mode='lines+markers', name='Recall'))
    fig.add_trace(go.Scatter(x=df_scores.index, y=df_scores['f'], mode='lines+markers', name='F1 Score'))

    fig.update_layout(
        title="ROUGE Scores for Each Row",
        xaxis_title="Row Index",
        yaxis_title="Score"
    )
    fig.show()

### Hugging Face models wrappers

The following code template showcases how to wrap a Hugging Face model for compatibility with the ValidMind Developer Framework. We will load an example model using the transformers API and then run some predictions on our test dataset.

The ValidMind developer framework provides support for Hugging Face transformers out of the box, so in the following section we will show how to initialize multiple transformers models with the `init_model` function, removing the need for a custom wrapper. In cases where you need extra pre-processing or post-processing steps, you can use the following code template as a starting point to wrap your model.

In [18]:
from dataclasses import dataclass
import pandas as pd
from transformers import pipeline

@dataclass
class AbstractSummarization_HuggingFace:
    """
    A VM Model instance wrapper for abstract summarization using HuggingFace Transformers.
    """
    model: any
    tokenizer: any
    predicted_prob_values: list = None

    def __init__(self, model_name=None, model=None, tokenizer=None):
        pipeline_task = "summarization"
        self.model_name = model_name
        self.pipeline_task = pipeline_task
        self.model = pipeline(pipeline_task, model=model, tokenizer=tokenizer)

    def predict(self, texts, params={}):
        """
        Generates summaries for the given texts.
        
        Parameters:
        - texts (list): List of texts to be summarized.
        - params (dict, optional): Dictionary that may contain "min_length" and/or "max_length" to control the produced summary's length.
        
        Returns:
        - List of summaries.
        """
        
        min_length = params.get("min_length")
        max_length = params.get("max_length")
        
        # If either value is None, don't pass it to the model function
        model_args = {}
        if min_length is not None:
            model_args["min_length"] = min_length
        if max_length is not None:
            model_args["max_length"] = max_length

        summaries = []

        for text in texts:
            data = [str(text)]
            results = self.model(data, **model_args)  # Using ** unpacking to pass arguments conditionally
            results_df = pd.DataFrame(results)
            summary = results_df["summary_text"].values[0] if "summary_text" in results_df.columns else results_df["label"].values[0]
            summaries.append(summary)

        return summaries

    def predict_proba(self):
        """
        Retrieves predicted probabilities after prediction. 
        Note: Not all models provide predicted probabilities.
        """
        if self.predicted_prob_values is None:
            raise ValueError("First run predict method to retrieve predicted probabilities")
        return self.predicted_prob_values

    def description(self):
        """
        Describes the methods available in the class.
        
        The class provides methods for abstract summarization using HuggingFace Transformers.
        
        The predict method:
        1. Generates summaries for given texts.
        2. Accepts a 'params' dictionary which can contain optional 'min_length' and 'max_length' parameters to control the length of the produced summary.
        
        The predict_proba method:
        Retrieves predicted probabilities after prediction (if available by the model).
        """
        return self.description.__doc__


In [19]:
from dataclasses import dataclass
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

@dataclass
class ExtractiveSummarization_BERT:
    model: any
    tokenizer: any

    def _get_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding='max_length')
        with torch.no_grad():
            output = self.model(**inputs)
        return output['last_hidden_state'].mean(dim=1).squeeze().detach().numpy()

    def predict(self, texts, params={}):
        summaries = []

        # Extract summary_length from params, default to None if not provided
        summary_length = params.get("summary_length", None)

        for text in texts:
            sentences = text.split('. ')
            
            document_embedding = self._get_embedding(' '.join(sentences))
            sentence_embeddings = [self._get_embedding(sentence) for sentence in sentences]
            similarities = cosine_similarity(sentence_embeddings, [document_embedding])
            sorted_indices = np.argsort(similarities, axis=0)[::-1].squeeze()

            # Determine summary length
            if summary_length is None:  # If not provided, use 20% of the total sentence count
                top_k = int(len(sentences) * 0.2)
            else:  # If provided, use the user-defined length
                top_k = summary_length

            # Extract the top sentences based on the chosen summary length
            selected_sentences = [sentences[i] for i in sorted_indices[:top_k]]

            summaries.append(' '.join(selected_sentences))
        
        return summaries

    def description(self):
        """ 
        Provides a description of the methods available for extractive summarization with the current model.
        
        The model ranks sentences in the input text based on their similarity to the overall document meaning, as determined by BERT embeddings. The top-ranked sentences are selected to form the summary.
        
        The length of the summary can be controlled in two ways:
        1. Automatically: Where the model summarizes to approximately 20% of the original text's sentence count.
        2. User-defined: By specifying the 'summary_length' parameter in the 'params' dictionary when calling the 'predict' method.
        
        """
        return self.description.__doc__



## 2. Load Data

In this section, we'll load the financial dataset, which will be the foundation for our summarization analysis tasks. 

The dataset is structured with several columns, namely "ID", "topic", "input", and "reference_summary". Each record is identified by a unique "ID". The "topic" column specifies the category or theme of the news, the "input" column contains detailed articles or news content, and the "reference_summary" provides a concise summary of the respective article in the "input". The dataset sources its content from BBC news.

In [20]:
df_summarization = load_text_data(
    filepath='./datasets/bbc_text_cls_reference.csv', 
    num_records=5
)

display_nice(df_summarization, num_rows=2)

ID,topic,input,reference_summary
1,business,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high- speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding. Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. ""Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility,"" chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins. TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.","The firm, which is now one of the biggest investors in Google, benefited from sales of high- speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. Ad sales boost Time Warner profit"
2,business,"Dollar gains on Greenspan speech The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise. And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan's speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. ""I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time,"" said Robert Sinche, head of currency strateg

## 3. Extractive Summarization: Hugging Face-BERT Model

Extractive summarization is a technique used to produce a summary by selecting and extracting whole sentences or passages from the source document without modifying the original content. Essentially, it identifies and "extracts" the most important and relevant information from the larger text to create a condensed version.

In [21]:
model = BertModel.from_pretrained("bert-base-uncased")  
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
extractive_model = ExtractiveSummarization_BERT(model=model, tokenizer=tokenizer)

In [22]:
data = df_summarization.input.values.tolist()

params = {
    "summary_length": None,
}

extractive_summary = extractive_model.predict(data, params)

df_summarization = add_list_to_df(
    df = df_summarization, 
    column_data = extractive_summary,
    column_name = "extractive_summary"
)

display_nice(df_summarization, num_rows=2)

ID,topic,input,reference_summary,extractive_summary
1,business,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high- speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding. Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. ""Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility,"" chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins. TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.","The firm, which is now one of the biggest investors in Google, benefited from sales of high- speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. Ad sales boost Time Warner profit","Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high- speed internet connections and higher advert sales For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins. TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding. Time Warner's fourth quarter profits were slightly better than analysts' expectations"
2,business,"Dollar gains on Greenspan speech The dollar has hit its highest level agains

## 4. Abstract Summarization: Hugging Face-T5 Model

Abstractive summarization is a technique used to produce a summary by understanding the main ideas of a source document and then expressing those ideas in a concise manner using new words and sentences, often not present in the original text. Instead of just extracting and repurposing existing sentences from the document, as is done in extractive summarization, abstractive summarization aims to capture the essence or meaning of the content and convey it in a shorter form.

In [23]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
abstract_model = AbstractSummarization_HuggingFace(model=model, tokenizer=tokenizer)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [24]:
data = df_summarization.input.values.tolist()

params = {
    "summary_length": None
}

abstract_summary = abstract_model.predict(data, params)

df_summarization = add_list_to_df(
    df = df_summarization, 
    column_data = abstract_summary,
    column_name = "abstract_summary"
)

display_nice(df_summarization, num_rows=2)

Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors


ID,topic,input,reference_summary,extractive_summary,abstract_summary
1,business,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high- speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding. Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. ""Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility,"" chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins. TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.","The firm, which is now one of the biggest investors in Google, benefited from sales of high- speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. Ad sales boost Time Warner profit","Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high- speed internet connections and higher advert sales For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins. TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding. Time Warner's fourth quarter profits were slightly better than analysts' expectations",time Warner profits jumped 76% to $1.13bn (£600m) for the three months 

## 5. Validation

### Data Description Metrics

##### Text Description Table

- Total Words: Assess the length and complexity of the input text. Longer documents might require more sophisticated summarization techniques, while shorter ones may need more concise summaries.

- Total Sentences: Understand the structural makeup of the content. Longer texts with numerous sentences might require the model to generate longer summaries to capture essential information.

- Avg Sentence Length: Determine the average length of sentences in the text. This can help the model decide on the appropriate length for generated summaries, ensuring they are coherent and readable.

- Total Paragraphs: Analyze how the content is organized into paragraphs. The model should be able to maintain the logical structure of the content when producing summaries.

- Total Unique Words: Measure the diversity of vocabulary in the text. A higher count of unique words could indicate more complex content, which the model needs to capture accurately.

- Most Common Words: Identify frequently occurring words that likely represent key themes. The model should pay special attention to including these words and concepts in its summaries.

- Total Punctuations: Evaluate the usage of punctuation marks, which contribute to the tone and structure of the content. The model should be able to maintain appropriate punctuation in summaries.

- Lexical Diversity: Calculate the richness of vocabulary in relation to the overall text length. A higher lexical diversity suggests a broader range of ideas and concepts that the model needs to capture in its summaries.

In [25]:
params = {
    "text_column": "input",
    "unwanted_tokens": {'s', 's\'', 'mr', 'ms', 'mrs', 'dr', '\'s', ' ', "''", 'dollar', 'us', '``'},
    "lang": "english"
}

df_text_description = text_description_table(df_summarization, params)
display(df_text_description)

,Total Words,Total Sentences,Avg Sentence Length,Total Paragraphs,Total Unique Words,Total Punctuations,Lexical Diversity
0,490,20,21.1,6,187,63,0.7
1,433,15,25.6,4,163,35,0.8
2,294,12,22.0,4,116,24,0.7
3,472,19,21.4,6,203,53,0.8
4,306,12,22.1,4,132,38,0.8


##### Text Description Histogram

In [31]:
params = {
    "text_column": "input",
    "unwanted_tokens": {'s', 's\'', 'mr', 'ms', 'mrs', 'dr', '\'s', ' ', "''", 'dollar', 'us', '``'},
    "lang": "english"
}

text_description_histograms(df_summarization, params)

##### Text Description Scatter

In [35]:
# Define the combinations you want to plot
combinations_to_plot = [
    ("Total Words", "Total Sentences"),
    ("Total Words", "Total Unique Words"),
    ("Total Sentences", "Avg Sentence Length"),
    ("Total Unique Words", "Lexical Diversity")
]

params = {
    "combinations_to_plot": combinations_to_plot,
    "text_column": "input",
    "unwanted_tokens": {'s', 's\'', 'mr', 'ms', 'mrs', 'dr', '\'s', ' ', "''", 'dollar', 'us', '``'},
    "lang": "english"
}

text_description_scatter(df_summarization, params)

##### Text Structure Histogram

- Word Frequencies: This metric provides a histogram of how often words appear with a given frequency. For example, if a lot of words appear only once in a document, it might be indicative of a text rich in unique words. On the other hand, a small set of words appearing very frequently might indicate repetitive content or a certain theme or pattern in the text.

- Sentence Positions vs. Sentence Lengths: This bar chart showcases the length of each sentence (in terms of word count) in their order of appearance in the document. This can give insights into the flow of information in a text, highlighting any long, detailed sections or brief, potentially superficial areas.

- Sentence Lengths Distribution: A histogram showing the frequency of sentence lengths across the document. Long sentences might contain a lot of information but could be harder for summarization models to digest and for readers to comprehend. Conversely, many short sentences might indicate fragmented information.

- Word Lengths Distribution: A histogram of the lengths of words in the document. Extremely long words might be anomalies, technical terms, or potential errors in the corpus. Conversely, a majority of very short words might denote lack of depth or specificity.

In [36]:
params = {
    "text_column": 'input',
    "num_docs_to_plot": 2
}

text_structure_histograms(df_summarization, params)

### Model Performance Metrics

##### Token Disparity Histograms

In [ ]:
params = {
    "reference_column": 'reference_summary',
    "generated_column": 'summary_2'
}

token_disparity_histograms(df_summarization, params)

##### ROUGE-N Score

The ROUGE score ((Recall-Oriented Understudy for Gisting Evaluation) is a widely adopted set of metrics used for evaluating automatic summarization and machine translation. It fundamentally measures the overlap between the n-grams in the generated summary and those in the reference summary.

- ROUGE-N: This evaluates the overlap of n-grams between the produced summary and reference summary. It calculates precision (the proportion of n-grams in the generated summary that are also present in the reference summary), recall (the proportion of n-grams in the reference summary that are also present in the generated summary), and F1 score (the harmonic mean of precision and recall).

- ROUGE-L: This metric is based on the Longest Common Subsequence (LCS) between the generated and reference summaries. LCS measures the longest sequence of tokens in the generated summary that matches the reference, without considering the order. It's beneficial because it can identify and reward longer coherent matching sequences.

- ROUGE-S: This measures the skip-bigram overlap, considering the pair of words in order as "bigrams" while allowing arbitrary gaps or "skips". This can be valuable to capture sentence-level structure similarity.

In [ ]:
params = {
    "metric": "rouge-l",
    "ref_column": "reference_summary",
    "gen_column": "summary_2", 
}

rouge_scores_plot(df_summarization, params)


### Bias Metrics

### Toxicity Metrics

##### Toxicity Table

In [ ]:
# Use the function with the parameters
params = {
    "hf_toxicity_obj": toxicity,
    "text_columns": ["input", "reference_summary"],
    "max_toxicity_threshold": 0.7,
    "min_toxicity_threshold": 0.1
}

df_metric_results = hf_toxicity_table(df_summarization, params)
display_nice(df_metric_results, num_rows=2)

##### Toxicity Histograms

In [ ]:
params = {
    "hf_toxicity_obj": toxicity,
    "input_text_col": "input",
    "generated_text_col": "reference_summary"
}

hf_toxicity_histograms(df_summarization, params)

##### Toxicity Plots

In [ ]:
params = {
    "hf_toxicity_obj": toxicity,
    "input_text_col": "input",
    "generated_text_col": "reference_summary",
    "generated_score_threshold": 0
}
hf_toxicity_plot(df_summarization, params)

### Safety Metrics